In [3]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import time
import sys
import os

import seaborn as sns
import gensim
from collections import Counter
from operator import itemgetter

In [7]:
import django
import platform

print("platform: {}".format(platform.node()))

if platform.node() == "srv-mcc-apsis":
    sys.path.append('/home/muef/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/media/Data/MCC/tmv/BasicBrowser/')

#sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import *
from utils.tm_mgmt import update_topic_scores
from utils.text import *
from django.db.models import Q, Count, Func, F, Sum, Avg, Value as V

platform: srv-mcc-apsis


In [8]:
def get_frequency(run_id):
    topics = Topic.objects.filter(run_id=run_id)
    
    top_words_combined = []
    for tw in topics.values('top_words'):
        top_words_combined = top_words_combined + tw['top_words']
        
    word_count = Counter(top_words_combined)
    sorted_word_count = sorted(word_count.items(), key=itemgetter(1), reverse=True)

    return sorted_word_count, topics, word_count

In [9]:
def get_word_score(run_id):
    stat = RunStats.objects.get(run_id=run_id)
    K = stat.K
    
    freq, topics, word_count = get_frequency(run_id=run_id)
    word_count_list = word_count.items()
    
    score_dict = {}
    for word, count in word_count_list:
        word_share = count/K
        word_score = (count - 1)/(K-1)
        score_dict[word] = word_score
    
    # find mean score of each topic in one topic model
    # adding topics to list by topic
    sigma_mws = []
    sigma_ttw = []
    for topic in topics.values('top_words'):
        total_word_score = []
        ttw = []
        
        for word in topic['top_words']:
            wws = score_dict[word]
            total_word_score = total_word_score + [wws]
            ttw = ttw + [word]
    
        mean_word_score = sum(total_word_score)/K
        sigma_mws = sigma_mws + [mean_word_score]
        sigma_ttw = sigma_ttw + [ttw]
    
    return sigma_mws, sigma_ttw

In [10]:
stats = RunStats.objects.get(run_id=1038)

In [20]:
# topics classified by Anna as irrelevant
topic_ids = [132477, 132458, 132463, 132465, 132502, 132484, 132481, 132493, 132459, 132464, 132473,
             132494, 132470, 132482, 132467]
print(len(topic_ids))

topic_set = set()
for topic_id in topic_ids:
    topic = Topic.objects.get(id=topic_id)
    #print(topic.top_words)
    topic_set = (topic_set | set(topic.top_words))

print(list(topic_set))
df = pd.DataFrame(data=list(topic_set))

15
['saar', 'bundesrepubl', 'stimmt', 'ganz', 'fdp', 'ewg', 'fisch', 'schreib', 'bundeskanzl', 'gesetzentwurf', 'mull', 'drucksach', 'kolleg', 'franzos', 'deutschland', 'deutsch', 'bundesminist', 'gesagt', 'notwend', 'aufgab', 'bundesrat', 'abs', 'parlamentar', 'rat', 'bitt', 'tagesordnungspunkt', 'wunscht', 'zolltarif', 'abstimm', 'zuzustimm', 'mensch', 'vorschrift', 'ziff', 'abgeordnet', 'saargebiet', 'wirtschaft', 'entwurf', 'beantwort', 'land', 'altestenrat', 'ausschuss', 'bundesregier', 'federfuhr', 'berichterstatt', 'beschlussempfehl', 'fraktion', 'bundnis', 'moglich', 'schriftlich', 'ander', 'handzeich', 'kommission', 'merkel', 'minist', 'uberweisungsvorschlag', 'uberweis', 'welt', 'wer', 'spd', 'regel', 'grun', 'geht', 'verehrt', 'jahr', 'enthalt', 'kind', 'mocht', 'vervielfaltigt', 'verordn', 'massnahm', 'antwort', 'prasidentin', 'paris', 'vorlag', 'erst', 'tagesordn', 'sag', 'arndt', 'geschaftsbereich', 'vertrag', 'saarbevolker', 'herrn', 'anfrag', 'frankreich', 'gibt', 'beso

In [22]:
df.to_csv('potential_stopwords_from_irrelevant_topics_raw.csv')

In [32]:
df = pd.read_csv('potential_stopwords_from_irrelevant_topics.csv')

In [33]:
df

,#,words,include in list of extra stopwords
0,0,saar,n
1,1,bundesrepubl,n
2,2,stimmt,y
3,3,ganz,y
4,4,fdp,n
5,5,ewg,n
6,6,fisch,n
7,7,schreib,y
8,8,bundeskanzl,n
9,9,gesetzentwurf,n


In [37]:
extra_stopwords = df[df['include in list of extra stopwords'] == 'y']['words'].values
print(len(extra_stopwords))
print(extra_stopwords)

82
['stimmt' 'ganz' 'schreib' 'drucksach' 'kolleg' 'gesagt' 'notwend'
 'aufgab' 'parlamentar' 'rat' 'bitt' 'tagesordnungspunkt' 'wunscht'
 'abstimm' 'zuzustimm' 'mensch' 'vorschrift' 'ziff' 'abgeordnet'
 'beantwort' 'land' 'altestenrat' 'ausschuss' 'federfuhr' 'berichterstatt'
 'beschlussempfehl' 'fraktion' 'bundnis' 'moglich' 'schriftlich' 'ander'
 'handzeich' 'uberweisungsvorschlag' 'uberweis' 'welt' 'wer' 'regel'
 'geht' 'verehrt' 'jahr' 'enthalt' 'kind' 'mocht' 'vervielfaltigt'
 'verordn' 'massnahm' 'antwort' 'prasidentin' 'vorlag' 'erst' 'tagesordn'
 'sag' 'herrn' 'anfrag' 'gibt' 'besond' 'lieb' 'schon' 'umdruck'
 'gegenprob' 'angenomm' 'kollegin' 'antrag' 'bereit' 'berat' 'frau' 'jung'
 'betreff' 'brauch' 'dafur' 'word' 'eingebracht' 'ruf' 'uberwies' 'frag'
 'beschlussfass' 'bericht' 'glaub' 'dam' 'mehr' 'beantwortet' 'herr']


In [38]:
extra_stopwords_from_overlap = ["herr", "jahr", "mehr", "sag", "land", "schon", "neu", "ganz", "polit",
                   "heut", "antrag", "gibt", "kolleg", "geht", "berat", "frag", "mensch"]
print(len(extra_stopwords_from_overlap))

17


In [42]:
merged_sw_list = list(set(extra_stopwords) | set(extra_stopwords_from_overlap))
print(len(merged_sw_list))
print(merged_sw_list)

85
['stimmt', 'ganz', 'schreib', 'drucksach', 'kolleg', 'gesagt', 'notwend', 'aufgab', 'parlamentar', 'rat', 'heut', 'bitt', 'tagesordnungspunkt', 'wunscht', 'neu', 'abstimm', 'zuzustimm', 'mensch', 'abgeordnet', 'vorschrift', 'ziff', 'beantwort', 'land', 'altestenrat', 'ausschuss', 'federfuhr', 'berichterstatt', 'beschlussempfehl', 'fraktion', 'bundnis', 'moglich', 'schriftlich', 'ander', 'handzeich', 'uberweisungsvorschlag', 'uberweis', 'welt', 'wer', 'regel', 'geht', 'verehrt', 'jahr', 'enthalt', 'polit', 'kind', 'mocht', 'vervielfaltigt', 'verordn', 'massnahm', 'antwort', 'prasidentin', 'vorlag', 'erst', 'tagesordn', 'sag', 'herrn', 'anfrag', 'gibt', 'besond', 'lieb', 'schon', 'umdruck', 'gegenprob', 'angenomm', 'kollegin', 'antrag', 'bereit', 'berat', 'frau', 'jung', 'betreff', 'brauch', 'dafur', 'word', 'eingebracht', 'ruf', 'uberwies', 'frag', 'beschlussfass', 'bericht', 'glaub', 'dam', 'mehr', 'beantwortet', 'herr']
